In [1]:
import requests
import pandas as pd
from datetime import datetime
import pandas as pd

In [107]:
#parcel_number = '871288650'
parcel_number = '888058983'
query = f"""
SELECT opp.parcel_number, year_built, year_built_estimate, 
mailing_street, mailing_address_1,
building_code_description, category_code_description, 
n_violations, n_violations_open, n_violations_dangerous,
has_active_rental_license, expiration_date,
r1.recording_date, r1.grantees, r1.grantors, r1.cash_consideration, r1.property_count,
a1.market_value as market_value_latest_assessment, a1.latest_assessment_year
FROM opa_properties_public opp
LEFT JOIN (
    SELECT '{parcel_number}' as parcel_number, count(*) as n_violations
    FROM violations 
    WHERE opa_account_num = '{parcel_number}'
    
) v1
ON v1.parcel_number = opp.parcel_number
LEFT JOIN (
    SELECT '{parcel_number}' as parcel_number, count(*) as n_violations_open
    FROM violations 
    WHERE opa_account_num = '{parcel_number}'
    AND violationstatus = 'OPEN'
    
) v2
ON v1.parcel_number = v2.parcel_number
LEFT JOIN (
    SELECT '{parcel_number}' as parcel_number, count(*) as n_violations_dangerous
    FROM violations 
    WHERE opa_account_num = '{parcel_number}'
    AND caseprioritydesc in ('HAZARDOUS', 'UNSAFE', 'IMMINENTLY DANGEROUS')
    
) v3
ON v1.parcel_number = v3.parcel_number
LEFT JOIN (
    SELECT '{parcel_number}' as parcel_number, 
    CASE WHEN count(*) > 0 THEN TRUE ELSE FALSE END as has_active_rental_license, 
    max(expirationdate) as expiration_date
    FROM business_licenses 
    WHERE opa_account_num = '{parcel_number}'
    AND licensetype='Rental'
    AND licensestatus='Active'
) l1
ON v1.parcel_number = l1.parcel_number
LEFT JOIN (
    SELECT opa_account_num as parcel_number, 
    *
    FROM rtt_summary 
    WHERE opa_account_num = '{parcel_number}'
    AND (
        document_type='DEED' OR
        document_type='DEED SHERIFF' OR
        document_type='DEED OF CONDEMNATION' OR
        document_type='DEED LAND BANK'
    )
    ORDER BY recording_date LIMIT 1
) r1
ON v1.parcel_number = r1.parcel_number
LEFT JOIN (
    SELECT parcel_number, 
    year as latest_assessment_year,
    market_value
    FROM assessments a
    WHERE parcel_number = '{parcel_number}'
    ORDER BY year DESC LIMIT 1
) a1
ON v1.parcel_number = a1.parcel_number
WHERE opp.parcel_number = '{parcel_number}'

"""

df_json = requests.get(
    f"""
    https://phl.carto.com/api/v2/sql?q={query}
    """
).json()

if 'rows' in df_json:
    results = df_json["rows"]
else:
    print(df_json)

In [139]:
#parcel_number = '871288650'
parcel_number = '888058983'
query = f"""
    SELECT opa_account_num as parcel_number, 
    grantees, grantors, cash_consideration, recording_date, property_count, condo_name
    FROM rtt_summary 
    WHERE opa_account_num = '{parcel_number}'
    AND (
        document_type='DEED' OR
        document_type='DEED SHERIFF' OR
        document_type='DEED OF CONDEMNATION' OR
        document_type='DEED LAND BANK'
    )
    ORDER BY recording_date DESC

"""

df_json = requests.get(
    f"""
    https://phl.carto.com/api/v2/sql?q={query}
    """
).json()

if 'rows' in df_json:
    results = df_json["rows"]
else:
    print(df_json)
results

[{'parcel_number': '888058983',
  'grantees': '14 WA PARTNERS LP',
  'grantors': '1401 WALNUT STREET UNIT B LLC',
  'cash_consideration': 14000000,
  'recording_date': '2013-06-27T00:00:00Z',
  'property_count': 7,
  'condo_name': '1401 WALNUT TOWER'},
 {'parcel_number': '888058983',
  'grantees': '1401 WALNUT STREET UNIT B LLC',
  'grantors': 'K HOVNANIAN AT BROAD AND WALNUT LLC',
  'cash_consideration': 20000000,
  'recording_date': '2007-08-22T00:00:00Z',
  'property_count': 8,
  'condo_name': 'WALNUT TOWERS'},
 {'parcel_number': '888058983',
  'grantees': 'K HOVNANIAN AT BROAD AND WALNUT LLC',
  'grantors': '1401 WALNUT PARTNERS L P',
  'cash_consideration': 1450000,
  'recording_date': '2006-06-29T00:00:00Z',
  'property_count': 2,
  'condo_name': 'WALNUT TOWERS'}]

In [40]:
# Question: How does the opa_properties_public.owner_1/owner_2 relate to the rtt_summary.grantees?
import requests
import pandas as pd
query = """
SELECT owner_1, owner_2, grantees, rtt_summary.recording_date, opa.recording_date as recording_date2,

rtt_summary.opa_account_num, parcel_number,
opa_account_num = parcel_number as is_same
from opa_properties_public opa
JOIN rtt_summary
ON (rtt_summary.grantees like concat(owner_1,';%') OR rtt_summary.grantees = owner_1)
AND rtt_summary.opa_account_num = parcel_number
WHERE owner_1 is not null
AND    (document_type='DEED' OR
        document_type='DEED SHERIFF' OR
        document_type='DEED OF CONDEMNATION' OR
        document_type='DEED LAND BANK')
AND rtt_summary.grantees not like CASE WHEN owner_2 is null THEN owner_1 ELSE concat(owner_1, '; ', owner_2) END 
AND rtt_summary.grantees not like CASE WHEN owner_2 is null THEN owner_1 ELSE concat(owner_1, '; ', owner_2,';%') END 
AND rtt_summary.grantees != owner_1
--AND rtt_summary.grantees not like concat(owner_1,';%')
LIMIT 10
"""
df_json = requests.get(
    f"""
    https://phl.carto.com/api/v2/sql?q={query}
    """
).json()

if 'rows' in df_json:
    results = df_json["rows"]
else:
    print(df_json)
results

[{'owner_1': 'CHEN YANWEI S',
  'owner_2': None,
  'grantees': 'CHEN YANWEI S; CHEN YU FANG',
  'recording_date': '2004-06-07T00:00:00Z',
  'recording_date2': '2006-04-27T00:00:00Z',
  'opa_account_num': '542181300',
  'parcel_number': '542181300',
  'is_same': True},
 {'owner_1': 'ZALE JAMES J',
  'owner_2': None,
  'grantees': 'ZALE JAMES J; ZALE LORETTA',
  'recording_date': '2004-06-07T00:00:00Z',
  'recording_date2': '2014-09-15T00:00:00Z',
  'opa_account_num': '631296400',
  'parcel_number': '631296400',
  'is_same': True},
 {'owner_1': 'TALLEY JAMES E',
  'owner_2': 'TALLEY JEANETTE',
  'grantees': 'TALLEY JAMES E; TALLEY LOUISE',
  'recording_date': '2004-06-07T00:00:00Z',
  'recording_date2': '2012-12-04T00:00:00Z',
  'opa_account_num': '033185500',
  'parcel_number': '033185500',
  'is_same': True},
 {'owner_1': 'GALLAGHER ANDREW',
  'owner_2': None,
  'grantees': 'GALLAGHER ANDREW; GALLAGHER LEISA',
  'recording_date': '2004-06-07T00:00:00Z',
  'recording_date2': '2008-08-26

In [61]:
# Question: How does the opa_properties_public.owner_1/owner_2 relate to the rtt_summary.grantees?
# Looks like you can join on recording date to get what you want!!!
import requests
import pandas as pd
query = """
SELECT mailing_street, mailing_address_1, owner_1, owner_2, grantees, rtt_summary.recording_date, opa.recording_date as recording_date2,
rtt_summary.opa_account_num, parcel_number,
opa_account_num = parcel_number as is_same
from opa_properties_public opa
JOIN rtt_summary
ON rtt_summary.recording_date =  opa.recording_date
AND rtt_summary.opa_account_num = opa.parcel_number
WHERE (document_type='DEED' OR
        document_type='DEED SHERIFF' OR
        document_type='DEED OF CONDEMNATION' OR
        document_type='DEED LAND BANK')
and parcel_number = '881028600'
"""
df_json = requests.get(
    f"""
    https://phl.carto.com/api/v2/sql?q={query}
    """
).json()

if 'rows' in df_json:
    results = df_json["rows"]
else:
    print(df_json)
results

[{'mailing_street': '319 WOODLEY RD',
  'mailing_address_1': None,
  'owner_1': '2215 WALNUT STREET ASSOCI',
  'owner_2': None,
  'grantees': '2215 WALNUT STREET ASSOCIATES LP',
  'recording_date': '2007-10-16T00:00:00Z',
  'recording_date2': '2007-10-16T00:00:00Z',
  'opa_account_num': '881028600',
  'parcel_number': '881028600',
  'is_same': True},
 {'mailing_street': '319 WOODLEY RD',
  'mailing_address_1': None,
  'owner_1': '2215 WALNUT STREET ASSOCI',
  'owner_2': None,
  'grantees': '2215 WALNUT STREET ASSOCIATES LP',
  'recording_date': '2007-10-16T00:00:00Z',
  'recording_date2': '2007-10-16T00:00:00Z',
  'opa_account_num': '881028600',
  'parcel_number': '881028600',
  'is_same': True}]

In [23]:
# Question: Can we get all the grantees for a given mailing address, then get all the properties that they own?

# 1. All properties with a given mailing address/street

'''
Join to the rtt_summary but where it matches grantors and the recording date is greater than
the one that matches the opp -> or maybe just return all the ones that are greater than that recording
date rather than the join, then we can get violations/complaints since that recording date basically

'''
import requests
import pandas as pd

mailing_address_subquery = """
opa_account_num = '291284800'
"""

mailing_address_subquery = """
opa_account_num = '881028600'
"""

query = f"""
SELECT rtt_w_mailing.* FROM 
opa_properties_public opa JOIN 
(
    SELECT rtt_summary.*, owner_1, owner_2,
    mailing_street, mailing_address_1, mailing_address_2, mailing_care_of
    from opa_properties_public opa
    JOIN rtt_summary
    ON rtt_summary.recording_date =  opa.recording_date
    AND rtt_summary.opa_account_num = opa.parcel_number
    WHERE (document_type='DEED' OR
            document_type='DEED SHERIFF' OR
            document_type='DEED OF CONDEMNATION' OR
            document_type='DEED LAND BANK')
    AND {mailing_address_subquery}
) rtt_w_mailing
ON (
    (opa.mailing_street is null and rtt_w_mailing.mailing_street is null)
    OR opa.mailing_street = rtt_w_mailing.mailing_street
)
AND (
    (opa.mailing_address_1 is null and rtt_w_mailing.mailing_address_1 is null) 
    OR opa.mailing_address_1 = rtt_w_mailing.mailing_address_1
)
"""


query = f"""


 SELECT rtt_summary.grantees, rtt_summary2.grantors, rtt_summary.recording_date as buy_date,
 rtt_summary2.recording_date as sell_date,
 opa_account_num,
 concat_ws(' ', address_low, address_low_suffix, address_low_frac, address_high, street_predir, rtt_summary.street_name, street_suffix, street_postdir) as address
 , mailing_street, mailing_address_1, mailing_address_2, mailing_city_state, mailing_zip, mailing_care_of
    from 
    opa_properties_public opa
    -- get when it was bought by joining on grantees
    JOIN (SELECT * FROM rtt_summary  WHERE (document_type='DEED' OR
            document_type='DEED SHERIFF' OR
            document_type='DEED OF CONDEMNATION' OR
            document_type='DEED LAND BANK')
        ) rtt_summary
    ON rtt_summary.recording_date =  opa.recording_date
    AND rtt_summary.opa_account_num = opa.parcel_number
    -- get when it was sold by joining the grantee to the grantor
    LEFT JOIN (SELECT grantors, recording_date FROM rtt_summary  WHERE (document_type='DEED' OR
            document_type='DEED SHERIFF' OR
            document_type='DEED OF CONDEMNATION' OR
            document_type='DEED LAND BANK')
        ) rtt_summary2
    ON rtt_summary.grantees = rtt_summary2.grantors
    WHERE (rtt_summary.recording_date < rtt_summary2.recording_date OR rtt_summary2.recording_date is null)
    AND {mailing_address_subquery}
"""



df_json = requests.get(
    f"""
    https://phl.carto.com/api/v2/sql?q={query}
    """
).json()

if 'rows' in df_json:
    results = df_json["rows"]
else:
    print(df_json)
results

[{'grantees': '2215 WALNUT STREET ASSOCIATES LP',
  'grantors': None,
  'buy_date': '2007-10-16T00:00:00Z',
  'sell_date': None,
  'opa_account_num': '881028600',
  'address': '135 37 S 23RD ST',
  'mailing_street': '319 WOODLEY RD',
  'mailing_address_1': None,
  'mailing_address_2': None,
  'mailing_city_state': 'MERION STATION PA',
  'mailing_zip': '19066',
  'mailing_care_of': None},
 {'grantees': '2215 WALNUT STREET ASSOCIATES LP',
  'grantors': None,
  'buy_date': '2007-10-16T00:00:00Z',
  'sell_date': None,
  'opa_account_num': '881028600',
  'address': '2215 WALNUT ST',
  'mailing_street': '319 WOODLEY RD',
  'mailing_address_1': None,
  'mailing_address_2': None,
  'mailing_city_state': 'MERION STATION PA',
  'mailing_zip': '19066',
  'mailing_care_of': None}]

In [21]:
pd.DataFrame(results)

,cartodb_id,the_geom,the_geom_webmercator,objectid,document_id,document_type,display_date,street_address,zip_code,ward,...,opa_account_num,legal_remarks,discrepancy,property_count,owner_1,owner_2,mailing_street,mailing_address_1,mailing_address_2,mailing_care_of
0,1029008,0101000020E61000004022096069CB52C0FE9688E8C5F9...,0101000020110F000097B4B7B3AAEC5FC121335DDBFE88...,8769608,51792648,DEED,2007-09-28T00:00:00Z,2215 WALNUT ST,19103,8,...,881028600,None,no,2,2215 WALNUT STREET ASSOCI,None,319 WOODLEY RD,None,None,None
1,1029009,0101000020E61000004022096069CB52C0FE9688E8C5F9...,0101000020110F000097B4B7B3AAEC5FC121335DDBFE88...,8769609,51792648,DEED,2007-09-28T00:00:00Z,2215 WALNUT ST,19103,8,...,881028600,None,no,2,2215 WALNUT STREET ASSOCI,None,319 WOODLEY RD,None,None,None


In [118]:
# Question 2: How many account numbers have multiple addresses?


mailing_address_subquery = """
opa_account_num = '881028600'
"""
#  {'street_address': '1514 S BROAD ST', 'address': '1734 MOUNT VERNON ST'},
#  {'street_address': '1030 FEDERAL ST', 'address': '1015 LATONA ST'},
#  {'street_address': '3101 N BROAD ST', 'address': '1345 W CLEARFIELD ST'},


query = f"""
SELECT * FROM (
SELECT rtt_summary.street_address,
concat_ws(' ', address_low, 
address_low_suffix, address_low_frac, address_high, 
street_predir, rtt_summary.street_name, street_suffix, street_postdir) as address
FROM rtt_summary JOIN (

 SELECT parcel_number, count(*) as n_addresses
    from 
    opa_properties_public opa
     JOIN rtt_summary
    ON rtt_summary.recording_date =  opa.recording_date
    AND rtt_summary.opa_account_num = opa.parcel_number
    WHERE (document_type='DEED' OR
            document_type='DEED SHERIFF' OR
            document_type='DEED OF CONDEMNATION' OR
            document_type='DEED LAND BANK')
   GROUP BY parcel_number
) opa
ON opa.parcel_number = rtt_summary.opa_account_num
where n_addresses > 1
) opa
where (replace(address,'-',' ') not like concat(street_address,'%') 
and replace(street_address,'-',' ') not like concat(address,'%'))


"""
df_json = requests.get(
    f"""
    https://phl.carto.com/api/v2/sql?q={query}
    """
).json()

if 'rows' in df_json:
    results = df_json["rows"]
else:
    print(df_json)
results

[{'street_address': '1956 BRUNNER ST', 'address': '5602 ROSEHILL ST'},
 {'street_address': '6723 WISSAHICKON AVE', 'address': '6709 WISSAHICKON AVE'},
 {'street_address': '3508 W ALLEGHENY AVE', 'address': '3502 W ALLEGHENY AVE'},
 {'street_address': '303-05 S CHADWICK ST', 'address': '303 5 S CHADWICK ST'},
 {'street_address': '1514 S BROAD ST', 'address': '1734 MOUNT VERNON ST'},
 {'street_address': '2841 BRIGHTON PL', 'address': '2841 BRIGHTON ST'},
 {'street_address': '2306-08 N LAWRENCE ST',
  'address': '2306 8 N LAWRENCE ST'},
 {'street_address': '2306-08 N LAWRENCE ST',
  'address': '2306 8 N LAWRENCE ST'},
 {'street_address': '201-03 RIPKA ST', 'address': '201 3 RIPKA ST'},
 {'street_address': '1030 FEDERAL ST', 'address': '1015 LATONA ST'},
 {'street_address': '332 S HICKS ST', 'address': '3009 POPLAR ST'},
 {'street_address': '1933 S 9TH ST', 'address': '4933 WALNUT ST'},
 {'street_address': '117 CHESTNUT ST', 'address': '117 21 CHESTNUT ST'},
 {'street_address': '2411 TREMO

In [ ]:
# Can we do auto-complete on this?

In [ ]:
# messy

In [6]:
# Question: FIguring out mailing-care-of
import requests
import pandas as pd

mailing_address_subquery = """
opa_account_num = '881028600'
"""
#  {'street_address': '1514 S BROAD ST', 'address': '1734 MOUNT VERNON ST'},
#  {'street_address': '1030 FEDERAL ST', 'address': '1015 LATONA ST'},
#  {'street_address': '3101 N BROAD ST', 'address': '1345 W CLEARFIELD ST'},

# Given a parcel number, get the latest grantees
# Get the business_name and legalname if the opa 

query = f"""
 SELECT
   mailing_care_of, mailing_street, mailing_address_1, mailing_address_2, owner_1, owner_2 from opa_properties_public
   where lower(mailing_care_of) like 'odin %'
    
"""
query = f"""

     SELECT
    rtt_summary.property_count,
    rtt_summary.grantees as likely_owner,
    rtt_summary2.grantees as sold_to,
    rtt_summary.recording_date as start_dt,
    rtt_summary2.recording_date as end_dt,
    opa.parcel_number as opa_account_num,
    location as opa_address,
    unit as opa_address_unit,
    concat_ws(
        ' ',
        address_low,
        address_low_suffix,
        address_low_frac,
        address_high,
        street_predir,
        rtt_summary.street_name,
        street_suffix,
        street_postdir
    ) as address,
    opa.location,
    ST_Y(opa.the_geom) AS lat, ST_X(opa.the_geom) AS lng,
    opa.unit,
    opa.mailing_street,
    opa.mailing_address_1,
    opa.mailing_address_2,
    opa.mailing_city_state,
    opa.mailing_zip,
    opa.mailing_care_of,
    opa.market_value,
    CASE WHEN (rtt_summary.opa_account_num is null) THEN 'opa_properties_public' ELSE 'rtt_summary' END as source
    FROM
    opa_properties_public opa
    -- get when it was bought by joining on grantees
    -- sometimes the deeds dont match. Making this a LEFT join may mean
    -- additional properties get pulled in
    LEFT JOIN (
        SELECT * FROM rtt_summary  WHERE (
    document_type='DEED' OR
    document_type='DEED SHERIFF' OR
    document_type='DEED OF CONDEMNATION' OR
    document_type='DEED LAND BANK'
)
    ) rtt_summary
    ON rtt_summary.recording_date =  opa.recording_date
    AND rtt_summary.opa_account_num = opa.parcel_number
    -- get when it was sold by joining the grantee to the grantor
    LEFT JOIN (
        SELECT opa_account_num, grantors, grantees, recording_date
        FROM rtt_summary  WHERE (
    document_type='DEED' OR
    document_type='DEED SHERIFF' OR
    document_type='DEED OF CONDEMNATION' OR
    document_type='DEED LAND BANK'
)
    ) rtt_summary2
    ON rtt_summary.grantees = rtt_summary2.grantors
    AND rtt_summary2.opa_account_num = opa.parcel_number
    AND rtt_summary.recording_date < rtt_summary2.recording_date

        SELECT mailing_street, mailing_address_1, mailing_address_2, mailing_care_of, owner_1, owner_2
FROM opa_properties_public
WHERE mailing_street like '1200 CALLOWHILL%' AND mailing_street != '1200 CALLOWHILL ST #302'
GROUP BY mailing_street, mailing_address_1, mailing_address_2 , mailing_care_of, owner_1, owner_2

    WHERE opa.category_code_description not in ('Commercial', 'Industrial','Vacant Land')
"""
df_json = requests.get(
    "https://phl.carto.com/api/v2/sql",params={'q':query}
).json()

if 'rows' in df_json:
    results = df_json["rows"]
else:
    print(df_json)
pd.DataFrame(results)

{'error': ['syntax error at or near "SELECT"']}


,mailing_care_of,mailing_street,mailing_address_1,mailing_address_2,owner_1,owner_2
0,ODIN PROPERTIES LLC,1200 CALLOWHILL ST,STE 403,None,JULIAN K & ADRIAN C PROPE,None
1,ODIN PROPERTIES LLC,1200 CALLOWHILL ST,STE 403,None,JULIAN K & ADRIAN C PROPE,None
2,ODIN PROPERTIES LLC,1200 CALLOWHILL ST,STE 403,None,MARCON HOLDINGS LLC,None
3,ODIN PROPERTIES LLC,1200 CALLOWHILL ST,STE 403,None,JULIAN K AND ADRIAN C PRO,None
4,ODIN PROPERTIES LLC,1200 CALLOWHILL ST,STE 403,None,JULIAN K & ADRIAN C PROPE,None
5,ODIN PROPERTIES,1200 CALLOWHILL ST,STE 403,None,J KRINSKY PROPERTIES LLC,None
6,ODIN PROPERTIES LLC,1200 CALLOWHILL ST,STE 403,None,JULIAN KRINSKY PROPERTIES,None
7,ODIN PROPERTIES LLC,1200 CALLOWHILL ST,STE 403,None,JULIAN KRINSKY PROPERTIES,None


In [53]:
# Question 2: How many account numbers have multiple addresses?
import requests
import pandas as pd

mailing_address_subquery = """
opa_account_num = '881028600'
"""
#  {'street_address': '1514 S BROAD ST', 'address': '1734 MOUNT VERNON ST'},
#  {'street_address': '1030 FEDERAL ST', 'address': '1015 LATONA ST'},
#  {'street_address': '3101 N BROAD ST', 'address': '1345 W CLEARFIELD ST'},

# Given a parcel number, get the latest grantees
# Get the business_name and legalname if the opa 

query = f"""
 SELECT
   location,unit,opa.recording_date,grantees,grantors,rtt_summary.recording_date as buy from opa_properties_public  opa
   LEFT JOIN rtt_summary
   ON rtt_summary.opa_account_num = parcel_number
   where parcel_number = '888058989' 
   and document_type like '%DEED%'
    
"""
df_json = requests.get(
    "https://phl.carto.com/api/v2/sql",params={'q':query}
).json()

if 'rows' in df_json:
    results = df_json["rows"]
else:
    print(df_json)
pd.DataFrame(results)

,location,unit,recording_date,grantees,grantors,buy
0,1401 WALNUT ST,B12,2013-06-27T00:00:00Z,1401 WALNUT TOWER CONDOMINIUM,1401 WALNUT PARTNERS L P,2005-11-29T00:00:00Z
1,1401 WALNUT ST,B12,2013-06-27T00:00:00Z,K HOVNANIAN AT BROAD AND WALNUT LLC,1401 WALNUT PARTNERS L P,2005-11-29T00:00:00Z
2,1401 WALNUT ST,B12,2013-06-27T00:00:00Z,1401 WALNUT PARTNERS L P; K HOVNANIAN AT BROAD...,1401 WALNUT PARTNERS L P; K HOVNANIAN AT BROAD...,2005-11-29T00:00:00Z
3,1401 WALNUT ST,B12,2013-06-27T00:00:00Z,1401 WALNUT STREET UNIT B LLC,K HOVNANIAN AT BROAD AND WALNUT LLC,2007-08-22T00:00:00Z
4,1401 WALNUT ST,B12,2013-06-27T00:00:00Z,PNC BANK NATIONAL ASSOCIATION,14 WA PARTNERS LP,2018-11-30T00:00:00Z
5,1401 WALNUT ST,B12,2013-06-27T00:00:00Z,MANUFACTURERS AND TRADERS TRUST COMPANY,14 WA PARTNERS LP,2013-06-27T00:00:00Z
6,1401 WALNUT ST,B12,2013-06-27T00:00:00Z,PNC BANK NATIONAL ASSOCATION,14 WA PARTNERS LP,2015-11-03T00:00:00Z


In [96]:
df = pd.DataFrame(results)
df[pd.notnull(df['sold_to'])]

,property_count,grantees,business_name,legalname,sold_to,buy_date,sell_date,opa_account_num,opa_address,opa_address_unit,address,mailing_street,mailing_address_1,mailing_address_2,mailing_city_state,mailing_zip,mailing_care_of


In [56]:
# Question 2: How many account numbers have multiple addresses?
import requests
import pandas as pd

mailing_address_subquery = """
opa_account_num = '881028600'
"""
#  {'street_address': '1514 S BROAD ST', 'address': '1734 MOUNT VERNON ST'},
#  {'street_address': '1030 FEDERAL ST', 'address': '1015 LATONA ST'},
#  {'street_address': '3101 N BROAD ST', 'address': '1345 W CLEARFIELD ST'},


query = f"""
  
      
     SELECT
    category_code_description,
    parcel_number,
    location as opa_address,
    unit as opa_address_unit,
    concat_ws(
        ' ',
        address_low,
        address_low_suffix,
        address_low_frac,
        address_high,
        street_predir,
        rtt_summary.street_name,
        street_suffix,
        street_postdir
    ) as address,
    opa.location,
    ST_Y(opa.the_geom) AS lat, ST_X(opa.the_geom) AS lng,
    opa.unit,
    opa.mailing_street,
    opa.mailing_address_1,
    opa.mailing_address_2,
    opa.mailing_city_state,
    opa.mailing_zip,
    opa.mailing_care_of
    from
    opa_properties_public opa
    -- get when it was bought by joining on grantees
    LEFT JOIN (
        SELECT * FROM rtt_summary  WHERE (
    document_type='DEED' OR
    document_type='DEED SHERIFF' OR
    document_type='DEED OF CONDEMNATION' OR
    document_type='DEED LAND BANK'
)
    ) rtt_summary
    ON rtt_summary.recording_date =  opa.recording_date
    AND rtt_summary.opa_account_num = opa.parcel_number
    -- get when it was sold by joining the grantee to the grantor
    JOIN (
            SELECT mailing_street as mailing_street_inner,
            mailing_address_1 as mailing_address_1_inner
             FROM opa_properties_public
            WHERE parcel_number = '888058983'
        ) opa2 ON opa.mailing_street = opa2.mailing_street_inner
        AND (
              opa.mailing_address_1 = opa2.mailing_address_1_inner
              OR (opa.mailing_address_1 is null and opa2.mailing_address_1_inner is null)
        )

    WHERE opa.category_code_description not in ('Commercial', 'Industrial','Vacant Land')
              """
df_json = requests.get(
    "https://phl.carto.com/api/v2/sql",params={'q':query}
).json()

if 'rows' in df_json:
    results = df_json["rows"]
else:
    print(df_json)
df1 = pd.DataFrame(results)

In [26]:
# Question 2: How many account numbers have multiple addresses?
import requests
import pandas as pd

mailing_address_subquery = """
opa_account_num = '881028600'
"""
#  {'street_address': '1514 S BROAD ST', 'address': '1734 MOUNT VERNON ST'},
#  {'street_address': '1030 FEDERAL ST', 'address': '1015 LATONA ST'},
#  {'street_address': '3101 N BROAD ST', 'address': '1345 W CLEARFIELD ST'},


query = f"""
        SELECT parcel_number,
        ST_Y(opa.the_geom) AS lat, ST_X(opa.the_geom) AS lng,
        opa.category_code_description,
        opa.market_value,
        location, unit, concat_ws(';',opa.owner_1,opa.owner_2) as owners, 'mailing_address' as relation, opa.mailing_street, opa.mailing_address_1
        FROM opa_properties_public opa INNER JOIN (
            SELECT distinct mailing_street, mailing_address_1, mailing_address_2, mailing_zip
            FROM opa_properties_public opa
            WHERE opa.parcel_number = '888058983'
            and mailing_street is not null
        ) opa_mailing_address
        ON opa.mailing_street = opa_mailing_address.mailing_street
        AND (opa.mailing_address_1 = opa_mailing_address.mailing_address_1 or opa_mailing_address.mailing_address_1 is null)
        AND (opa.mailing_address_2 = opa_mailing_address.mailing_address_2 or opa_mailing_address.mailing_address_2 is null)
        AND (opa.mailing_zip = opa_mailing_address.mailing_zip or opa_mailing_address.mailing_zip is null)
"""
df_json = requests.get(
    "https://phl.carto.com/api/v2/sql",params={'q':query}
).json()

if 'rows' in df_json:
    results = df_json["rows"]
else:
    print(df_json)
df2 = pd.DataFrame(results)

In [57]:
df1

,category_code_description,parcel_number,opa_address,opa_address_unit,address,location,lat,lng,unit,mailing_street,mailing_address_1,mailing_address_2,mailing_city_state,mailing_zip,mailing_care_of
0,Single Family,888000060,1200-02 WALNUT ST,2,1200 WALNUT ST,1200-02 WALNUT ST,39.948684,-75.160756,2,110 S 19TH ST,SUITE 300,None,PHILADELPHIA PA,19103,None
1,Single Family,888058983,1401 WALNUT ST,B4,1401 WALNUT ST,1401 WALNUT ST,39.949600,-75.164592,B4,110 S 19TH ST,SUITE 300,None,PHILADELPHIA PA,19103,None
2,Single Family,888058983,1401 WALNUT ST,B4,1401 WALNUT ST,1401 WALNUT ST,39.949600,-75.164592,B4,110 S 19TH ST,SUITE 300,None,PHILADELPHIA PA,19103,None
3,Single Family,888058983,1401 WALNUT ST,B4,1401 WALNUT ST,1401 WALNUT ST,39.949600,-75.164592,B4,110 S 19TH ST,SUITE 300,None,PHILADELPHIA PA,19103,None
4,Single Family,888058989,1401 WALNUT ST,B12,1401 WALNUT ST,1401 WALNUT ST,39.949600,-75.164592,B12,110 S 19TH ST,SUITE 300,None,PHILADELPHIA PA,19103,None
5,Single Family,888058989,1401 WALNUT ST,B12,1401 WALNUT ST,1401 WALNUT ST,39.949600,-75.164592,B12,110 S 19TH ST,SUITE 300,None,PHILADELPHIA PA,19103,None
6,Single Family,888000070,1200-02 WALNUT ST,4,1200 WALNUT ST,1200-02 WALNUT ST,39.948684,-75.160756,4,110 S 19TH ST,SUITE 300,None,PHILADELPHIA PA,19103,None
7,Single Family,888000070,1200-02 WALNUT ST,4,1200 WALNUT ST,1200-02 WALNUT ST,39.948684,-75.160756,4,110 S 19TH ST,SUITE 300,None,PHILADELPHIA PA,19103,None
8,Single Family,888000070,1200-02 WALNUT ST,4,1200 WALNUT ST,1200-02 WALNUT ST,39.948684,-75.160756,4,110 S 19TH ST,SUITE 300,None,PHILADELPHIA PA,19103,None
9,Single Family,888000070,1200-02 WALNUT ST,4,1200 WALNUT ST,1200-02 WALNUT ST,39.948684,-75.160756,4,110 S 19TH ST,SUITE 300,None,PHILADELPHIA PA,19103,None


In [9]:
df2[df2['relation']=='mailing_address'].sort_values('location')

,parcel_number,lat,lng,category_code_description,market_value,location,unit,owners,relation,mailing_street,mailing_address_1
9,888000070,39.948684,-75.160756,Single Family,1054200,1200-02 WALNUT ST,4,1200 WALNUT PARTNERS L P,mailing_address,110 S 19TH ST,SUITE 300
12,888000055,39.948684,-75.160756,Commercial,1113300,1200-02 WALNUT ST,1,1200 WALNUT PARTNERS L P,mailing_address,110 S 19TH ST,SUITE 300
11,888000060,39.948684,-75.160756,Single Family,1345600,1200-02 WALNUT ST,2,1200 WALNUT PARTNERS L P,mailing_address,110 S 19TH ST,SUITE 300
10,888000075,39.948684,-75.160756,Single Family,1054200,1200-02 WALNUT ST,5,1200 WALNUT PARTNERS L P,mailing_address,110 S 19TH ST,SUITE 300
16,888058983,39.949600,-75.164592,Single Family,1669600,1401 WALNUT ST,B4,14 WA PARTNERS LP,mailing_address,110 S 19TH ST,SUITE 300
17,888058989,39.949600,-75.164592,Single Family,1669600,1401 WALNUT ST,B12,14 WA PARTNERS LP,mailing_address,110 S 19TH ST,SUITE 300
0,888058999,39.949600,-75.164592,Commercial,541000,1401 WALNUT ST,C,1429 WALNUT ST RETAIL LLC,mailing_address,110 S 19TH ST,SUITE 300
4,882020850,39.949707,-75.165457,Commercial,2206000,1429-33 WALNUT ST,RETAIL,1429 WALNUT ST RETAIL LLC,mailing_address,110 S 19TH ST,SUITE 300
7,882336500,39.949600,-75.164592,Commercial,8362200,150 S BROAD ST,None,1401 WALNUT STREET LOWER,mailing_address,110 S 19TH ST,SUITE 300
6,882420100,39.949842,-75.166361,Commercial,1215900,1509 WALNUT ST,None,1509 WALNUT PARTNERS L P,mailing_address,110 S 19TH ST,SUITE 300
